In [1]:
import os
import requests
import io
import pydicom
http://localhost:8888/notebooks/githealth/notebooks/Post%20to%20DICOMweb.ipynb#
#see https://stackoverflow.com/questions/15746558/how-to-send-a-multipart-related-with-requests-in-python


In [2]:
from urllib3.filepost import encode_multipart_formdata, choose_boundary

def encode_multipart_related(fields, boundary=None):
    if boundary is None:
        boundary = choose_boundary()

    body, _ = encode_multipart_formdata(fields, boundary)
    content_type = str('multipart/related; boundary=%s' % boundary)

    return body, content_type

In [4]:
fileroot = "C:\\githealth\\dicom-samples\\visus.com\\case4\\"

filename = fileroot + 'case4a_001.dcm'

url = "https://dicom-server-instance.azurewebsites.net/studies"

with open(filename,'rb') as reader:
    rawfile = reader.read()

files = {'file': ('report.xls', rawfile, 'application/dicom')}

body, content_type = encode_multipart_related(fields = files)
    
headers = {'Accept':'application/dicom+json', "Content-Type":content_type}
print(headers)

r = requests.post(url, body, headers=headers, verify=False)


{'Accept': 'application/dicom+json', 'Content-Type': 'multipart/related; boundary=77d5309f468e5e1640631ab753a2204d'}


ConnectionError: HTTPSConnectionPool(host='dicom-server-instance.azurewebsites.net', port=443): Max retries exceeded with url: /studies (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000027A36A63E08>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:

print(r.status_code)
print(r.request.headers)
#print(r.request.body)

In [ ]:
def upload_dcm_file(server_url,filepath):
    with open(filepath,'rb') as reader:
        rawfile = reader.read()

    files = {'file': ('report.xls', rawfile, 'application/dicom')}

    body, content_type = encode_multipart_related(fields = files)
    
    headers = {'Accept':'application/dicom+json', "Content-Type":content_type}

    r = requests.post(url, body, headers=headers, verify=False)
    return r

In [ ]:
r = upload_dcm_file(url,'C:\\githealth\\dicom-samples\\visus.com\\case4\\case4a_002.dcm')
print(r.status_code)
print(r.request.headers)

In [ ]:
%timeit
import os
rootdir = "C:\\githealth\\dicom-samples\\"

for subdir, dirs, files in os.walk(rootdir):
    if '.git' in subdir:
        continue
    for file in files:
        if not '.dcm' in file:
            continue
        fname = os.path.join(subdir, file)
        print(fname)
        r = upload_dcm_file(url, fname)
        print(r.status_code)

In [ ]:
!pip install torch torchvision feather-format kornia pyarrow --upgrade

In [2]:
#!pip install nvidia-ml-py3
#!pip install pydicom kornia opencv-python scikit-image
#!pip install pyarrow

In [6]:
#!pip install git+https://github.com/fastai/fastai2
!pip install fastai2

In [2]:
#from fastai2 import *
#from fastai2.basics import *
from fastai2.medical.imaging import *
from fastai2.data.all import *

In [4]:
path = Path('C:\\githealth\\dicom-samples\\visus.com\\case4\\')
path_trn = path
fns_trn = path_trn.ls()
fns_trn[:5].attrgot('name')

(#5) ['case4a_001.dcm','case4a_002.dcm','case4b_001.dcm','case4b_002.dcm','case4c_001.dcm']

In [22]:
fn = fns_trn[0]
dcm = fn.dcmread()
dcm

(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M_SE', 'M', 'SE']
(0008, 0012) Instance Creation Date              DA: '20051102'
(0008, 0013) Instance Creation Time              TM: '115144'
(0008, 0014) Instance Creator UID                UI: 1.3.46.670589.11.5526.5
(0008, 0016) SOP Class UID                       UI: MR Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.276.0.50.192168001099.8687553.14547392.48
(0008, 0020) Study Date                          DA: '20010101'
(0008, 0021) Series Date                         DA: '20070511'
(0008, 0022) Acquisition Date                    DA: '20070511'
(0008, 0023) Content Date                        DA: '20070511'
(0008, 0030) Study Time                          TM: '114244'
(0008, 0031) Series Time                         TM: '114244'
(0008, 0032) Acquisition Time                    TM: '114244'
(0008, 0033) Content Tim

In [25]:
df_tst = pd.DataFrame.from_dicoms(fns_trn[2:3]) #, px_summ=True)

{'SpecificCharacterSet': 'ISO_IR 100', 'ImageType': 'ORIGINAL', 'InstanceCreationDate': '20051102', 'InstanceCreationTime': '115144', 'InstanceCreatorUID': '1.3.46.670589.11.5526.5', 'SOPClassUID': '1.2.840.10008.5.1.4.1.1.4', 'SOPInstanceUID': '1.2.276.0.50.192168001099.8687553.14547392.73', 'StudyDate': '20010101', 'SeriesDate': '20070511', 'AcquisitionDate': '20070511', 'ContentDate': '20070511', 'StudyTime': '114300', 'SeriesTime': '114300', 'AcquisitionTime': '114300', 'ContentTime': '114300', 'AccessionNumber': '11788765469991', 'Modality': 'MR', 'Manufacturer': 'Philips Medical Systems', 'InstitutionName': 'Anonymized Hospital', 'InstitutionAddress': 'Anonymized Address', 'ReferringPhysicianName': 'Dr. Anonymous', 'StationName': 'Any Station', 'StudyDescription': 'MRT Schädel', 'SeriesDescription': 'Brain      *T1/FFE/512', 'InstitutionalDepartmentName': '', 'AdmittingDiagnosesDescription': '', 'ManufacturerModelName': 'Intera', 'ReferencedPerformedProcedureStepSequence': (0008,

TypeError: __init__() missing 1 required positional argument: 'iterable'

In [14]:
df_tst.head()

""


In [24]:
# Cell
def _cast_dicom_special(x):
    cls = type(x)
    if not cls.__module__.startswith('pydicom'): return x
    if cls.__base__ == object: return x
    print('Hlello!')
    return cls #.__base__(iter(x))